In [1]:
%matplotlib inline
! realpath IKT450
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/MyDrive
! git clone https://github.com/JoarVarpe/IKT450.git
!nvidia-smi

from collections import Counter
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
import glob
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchvision

random.seed(7)

/content/IKT450
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 20201124_025526.jpg			     res_50_17.pt
 Candito+Linear+Program.pdf		     res_50_19.pt
 classifier0.pt				     res_50_1.pt
 classifier12.pt			     res_50_20.pt
 classifier13.pt			     res_50_25.pt
 classifier14.pt			     res_50_2.pt
 classifier16.pt			     res_50_30.pt
 classifier1.pt				     res_50_35.pt
 classifier2.pt				     res_50_38.pt
 classifier3.pt				     res_50_39.pt
 classifier4.pt				     res_50_3.pt
 classifier5.pt				     res_50_40.pt
 classifier6.pt				     res_50_45.pt
 classifier7.pt				     res_50_48.pt
 classifier_body0.pt			     res_50_49.pt
 classifier_body11.pt			     res_50_50.pt
 classifier_body1.pt			     res_50_55.pt
 classifier_body2.pt			     res_50_5.pt
 classifier_body3.pt			     res_50_60.pt
 classifier_body5.pt			     res_50_65.pt
'Colab Notebooks'			     res_50_70.pt
'Husleieavtale Joar Varpe 2-1 signert.pd

In [2]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
# model.fc = torch.nn.Linear( in_features=512,out_features=128) #res18
model.fc = torch.nn.Linear( in_features=2048,out_features=128) #res50
model


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
class Config():
    training_dir = "/content/IKT450/0_8/train/"
    testing_dir = "/content/IKT450/0_8/test/"
    training_dir_body = "/content/IKT450/0_8_body/train/"
    testing_dir_body = "/content/IKT450/0_8_body/test/"
    train_batch_size = 3
    train_number_epochs = 100

In [4]:
class SiameseNetworkDataset(Dataset):

    def __init__(self, imageFolderDataset, single_img=False, percent_same=None, transform=None, fixed=False):
        self.imageFolderDataset = imageFolderDataset
        self.transform = transform
        self.percent_same = percent_same
        self.single_img = single_img
        self.fixed = fixed
        print(self.percent_same)

        n_imgs = len(self)
        self.test_order = list(range(n_imgs))
        random.shuffle(self.test_order)

    def __getitem__(self, index):
        if self.single_img:
            return self.imageFolderDataset.imgs[index]
        else:
            # img0_tuple = random.choice(self.imageFolderDataset.imgs)
            img0_tuple = self.imageFolderDataset.imgs[index]
            if self.fixed:
                img1_tuple = self.imageFolderDataset.imgs[self.test_order[index]]
                img0 = Image.open(img0_tuple[0])
                img1 = Image.open(img1_tuple[0])

                if self.transform is not None:
                    img0 = self.transform(img0)
                    img1 = self.transform(img1)

                return img0, img1, img0_tuple, img1_tuple, torch.from_numpy(
                    np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))

            # we need to make sure approx 50% of images are in the same class
            if self.percent_same != None:
                should_get_same_class = self.percent_same
            else:

                should_get_same_class = random.randint(0, 1)

            if should_get_same_class:
                while True:
                    # keep looping till the same class image is found
                    img1_tuple = random.choice(self.imageFolderDataset.imgs)
                    if img0_tuple[1] == img1_tuple[1]:
                        break
            else:
                while True:
                    # keep looping till a different class image is found

                    img1_tuple = random.choice(self.imageFolderDataset.imgs)
                    if img0_tuple[1] != img1_tuple[1]:
                        break

            img0 = Image.open(img0_tuple[0])
            img1 = Image.open(img1_tuple[0])

            if self.transform is not None:
                img0 = self.transform(img0)
                img1 = self.transform(img1)

            return img0, img1, img0_tuple, img1_tuple, torch.from_numpy(
                np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))

    def __len__(self):
        return len(self.imageFolderDataset.imgs)


In [5]:
preprocess = transforms.Compose([
    
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
folder_dataset_train = dset.ImageFolder(root=Config.training_dir)
folder_dataset_test = dset.ImageFolder(root=Config.testing_dir)

In [7]:
siamese_dataset_train = SiameseNetworkDataset(imageFolderDataset=folder_dataset_train,
                                        transform=preprocess)
siamese_dataset_test = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
                                        transform=preprocess)

None
None


In [8]:
train_dataloader = DataLoader(siamese_dataset_train,
                        shuffle=True,
                        num_workers=8,
                        batch_size=Config.train_batch_size)

test_dataloader = DataLoader(siamese_dataset_test,
                        shuffle=True,
                        num_workers=8,
                        batch_size=Config.train_batch_size)



In [9]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        # d = euclidean_distance.detach().numpy()
        # if (d < 0).sum() > 0:
          # print("Got euclidean distance: ", d)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [10]:
import tqdm
criterion = ContrastiveLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.0005 )
model_save_name = 'res_50_88.pt' 
path = F"/content/gdrive/My Drive/{model_save_name}"
model.load_state_dict(torch.load(path))
model.train()
model.cuda()
best_loss = 1e10
for epoch in range(100):
    model.train()
    for batch in tqdm.tqdm(train_dataloader):
        img0, img1,  img0_info, img1_info, label = batch
        enc0 = model(img0.cuda())
        enc1 = model(img1.cuda())
        loss = criterion(enc0, enc1, label.cuda())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
  
    model.eval()
    total_loss = 0
    for i, data in enumerate(test_dataloader,0):
        with torch.no_grad():
            img0, img1 , img0_info, img1_info, label = data
            enc0 = model(img0.cuda())
            enc1 = model(img1.cuda())
            loss = criterion(enc0, enc1, label.cuda())
            total_loss += loss.item()
    if total_loss < best_loss:
        best_loss = total_loss
        print("saving model: ", epoch)
        print("best loss: ", best_loss)
        model_save_name = 'res_50_100+{}.pt'.format(epoch)
        path = F"/content/gdrive/MyDrive/{model_save_name}" 
        torch.save(model.state_dict(), path)
    elif epoch % 5 == 0:
        print("Saving model as epoch is: ", epoch)
        model_save_name = 'res_50_100+{}.pt'.format(epoch)
        path = F"/content/gdrive/MyDrive/{model_save_name}" 
        torch.save(model.state_dict(), path)
    print(total_loss)
  
  

print("training done!")


100%|██████████| 564/564 [00:58<00:00,  9.68it/s]


saving model:  0
best loss:  84.24524112812156


  0%|          | 0/564 [00:00<?, ?it/s]

84.24524112812156


100%|██████████| 564/564 [00:58<00:00,  9.63it/s]


...

84.87938176690362


  0%|          | 0/564 [00:00<?, ?it/s]

84.01639162024169


100%|██████████| 564/564 [00:57<00:00,  9.84it/s]


99.91030441823719
training done!


In [24]:
model_save_name = 'res_50_46.pt' 
path = F"/content/gdrive/My Drive/{model_save_name}"
model.load_state_dict(torch.load(path))
model.eval()
model.cuda()

folder_dataset_test = dset.ImageFolder(root=Config.testing_dir)
folder_dataset_test = dset.ImageFolder(root=Config.training_dir)
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
                                        transform=preprocess, fixed=True)
best_thresh = None
best_accuracy = -1



for test_it in range(30):
    counter = Counter()
    threshold = (test_it + 1) * 0.1
    test_dataloader = DataLoader(siamese_dataset,num_workers=6,batch_size=1,shuffle=False)
    dataiter1 = iter(test_dataloader)

    correct = 0
    wrong = 0
    for i in range(len(dataiter1)):
        x0,x1,tt0,tt1, label = next(dataiter1)
        
        with torch.no_grad():
            enc0 = model(x0.cuda())
            enc1 = model(x1.cuda())
            euclidean_distance = F.pairwise_distance(enc0, enc1)

        the_distance = euclidean_distance.item()


        if the_distance > threshold:
            same = 0
        else:
            same = 1

        if same == 0 and label.item() == 0:
          counter["TN"] += 1
        elif same == 0 and label.item() == 1:
          counter["FN"] += 1
        elif same == 1 and label.item() == 0:
          counter["FP"] += 1
        else:
          counter["TP"] += 1

        if same == label.item():
            correct += 1
        else:
            wrong += 1
    acc = correct / (correct + wrong)
    print("Th: {:.2f}  Accuracy: {:.4f}".format(threshold, acc))
    print(counter)
    if acc > best_accuracy:
        print("\tnew best acc!!!!")
        best_accuracy = acc
        best_thresh = threshold
        
        
          

print("the best threshold is:", best_thresh, "with accuracy: ", best_accuracy)


None
Th: 0.10  Accuracy: 0.0024
Counter({'FN': 1686, 'TN': 4})
	new best acc!!!!
Th: 0.20  Accuracy: 0.0024
Counter({'FN': 1686, 'TN': 4})
Th: 0.30  Accuracy: 0.0030
Counter({'FN': 1685, 'TN': 4, 'TP': 1})
	new best acc!!!!
Th: 0.40  Accuracy: 0.0065
Counter({'FN': 1679, 'TP': 7, 'TN': 4})
	new best acc!!!!
Th: 0.50  Accuracy: 0.0219
Counter({'FN': 1653, 'TP': 33, 'TN': 4})
	new best acc!!!!
Th: 0.60  Accuracy: 0.0367
Counter({'FN': 1628, 'TP': 58, 'TN': 4})
	new best acc!!!!
Th: 0.70  Accuracy: 0.0562
Counter({'FN': 1594, 'TP': 92, 'TN': 3, 'FP': 1})
	new best acc!!!!
Th: 0.80  Accuracy: 0.0929
Counter({'FN': 1531, 'TP': 155, 'FP': 2, 'TN': 2})
	new best acc!!!!
Th: 0.90  Accuracy: 0.1284
Counter({'FN': 1471, 'TP': 215, 'FP': 2, 'TN': 2})
	new best acc!!!!
Th: 1.00  Accuracy: 0.1840
Counter({'FN': 1377, 'TP': 309, 'FP': 2, 'TN': 2})
	new best acc!!!!
Th: 1.10  Accuracy: 0.2379
Counter({'FN': 1286, 'TP': 400, 'FP': 2, 'TN': 2})
	new best acc!!!!
Th: 1.20  Accuracy: 0.2870
Counter({'FN'

In [14]:
!nvidia-smi

Wed Dec  2 12:50:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    35W /  70W |   2293MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
!ps -aux|grep python

root          48  1.2  0.4 195436 61496 ?        Sl   10:04   2:06 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --LargeFileManager.delete_to_trash=False --MappingKernelManager.root_dir="/content"
root         258  0.0  0.0  18384  3028 ?        S    10:05   0:00 bash -c tail -n +0 -F "/root/.config/Google/DriveFS/Logs/drive_fs.txt" | python3 /opt/google/drive/drive-filter.py > "/root/.config/Google/DriveFS/Logs/timeouts.txt" 
root         260  0.0  0.0  29216  9360 ?        S    10:05   0:00 python3 /opt/google/drive/drive-filter.py
root        2556  0.0  0.0      0     0 ?        Z    10:48   0:02 [python3] <defunct>
root        2557  0.0  0.0      0     0 ?        Z    10:48   0:02 [python3] <defunct>
root        2558  0.0  0.0      0     0 ?        Z    10:48   0:02 [python3] <defunct>
root        2559  0.0  0.0      0     0 ?        Z    10:48   0:02 [python3] <defunct>
root        2560  0.0  0.0      0     0 ?   

In [ ]:
!kill 2603
